In [53]:
import threading
import pandas as pd
import numpy as np
import time as t
from datetime import datetime

In [54]:
def stream():
    global n
    while(n<100):
        print('thread1',datetime.now())
        t.sleep(0.5)
        n=n+1
def calc():
    global n
    while(n<100):
        print('thread2',datetime.now())
        t.sleep(0.5)
        n=n+1

In [55]:
n=0
thread1=threading.Thread(target=stream)
thread2=threading.Thread(target=calc)
e=threading.Event()
thread1.start()
thread2.start()
#thread1.join()
#thread2.join()

thread1 2021-03-02 11:47:54.496421
thread2 2021-03-02 11:47:54.497422
thread1 2021-03-02 11:47:54.997157
thread2 2021-03-02 11:47:54.998248
thread1 2021-03-02 11:47:55.498036
thread2 2021-03-02 11:47:55.499375
thread1 2021-03-02 11:47:55.999530
thread2 2021-03-02 11:47:56.001139
thread1 2021-03-02 11:47:56.501273
thread2 2021-03-02 11:47:56.502658
thread1 2021-03-02 11:47:57.001928
thread2 2021-03-02 11:47:57.003270
thread1 2021-03-02 11:47:57.502761
thread2 2021-03-02 11:47:57.504066
thread1 2021-03-02 11:47:58.003718
thread2 2021-03-02 11:47:58.005217
thread1 2021-03-02 11:47:58.504547
thread2 2021-03-02 11:47:58.506499
thread1 2021-03-02 11:47:59.006196
thread2 2021-03-02 11:47:59.007621
thread1 2021-03-02 11:47:59.506907thread2 2021-03-02 11:47:59.508306

thread2 2021-03-02 11:48:00.009271
thread1 2021-03-02 11:48:00.009346
thread2 2021-03-02 11:48:00.510223
thread1 2021-03-02 11:48:00.511414
thread1 2021-03-02 11:48:01.012336
thread2 2021-03-02 11:48:01.013465
thread1 2021-03-02 1

In [1]:
import hmac
import hashlib
import base64
import json
import time
import requests

# Enter your API Key and Secret here. If you don't have one, you can generate it from the website.
key = "b6c8818a4428e3f449bc738478589104e46f9f0698e97bb2"
secret = "730e2d8484103bcbfcb9b3e73b9b11b1e2f0587f73a0217ff136f309580e529f"

# python3
secret_bytes = bytes(secret, encoding='utf-8')
# python2
#secret_bytes = bytes(secret)

# Generating a timestamp
timeStamp = int(round(time.time() * 1000000000))
body = {
    "timestamp": timeStamp
}

json_body = json.dumps(body, separators = (',', ':'))

signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

headers = {
    'Content-Type': 'application/json',
    'X-AUTH-APIKEY': key,
    'X-AUTH-SIGNATURE': signature
}


In [2]:
import pandas as pd
import requests
import hmac
import hashlib
import base64
import json
import time
import numpy as np
from datetime import datetime
import requests # Install requests module first.
from datetime import datetime
import time as t
import threading
candle=pd.DataFrame(columns=['open','high','low','close'],index=[0])

In [3]:
def insert():
    url = "https://api.coindcx.com/exchange/ticker"
    coin='LINKBTC'
    global candle
    while(True):
        minute=datetime.now().minute
        price=[]
        while(minute==datetime.now().minute):
            try:
                response = requests.get(url)
                data = response.json()
                price.append(float(pd.DataFrame(data).query("market=='{0}'".format(coin))['last_price'].values[0]))
            except json.decoder.JSONDecodeError:
                continue
        candle=pd.concat([candle,pd.DataFrame({'open':price[0],'high':max(price),'low':min(price),'close':price[-1]},index=[0])],axis=0)
        print('thread1')
        thread2.start()
        #check len 180 values
        #if yes then set flag for calculation

In [4]:
def arbitrage():
    #find moving average and place order
    global candle,buy,mean,price,total
    print('thread2')
    value=candle.mean().sum()/4
    close=candle.tail(1)['close'].values[0]
    if (mean<value) & (buy==0):
        price=close
        print('buy at',price,datetime.now())
        mean=value
        buy=1
    elif ((mean>value) or (close<mean)) & (buy==1):
        total=price-close
        print('sell at',total,datetime.now())
        #sell
        buy=0
        mean=value
    candle=candle[1:]
    print('change length of the candle is',candle)
    #delete the first row in candle

In [5]:
mean=0
buy=0
price=0
total=0
thread1=threading.Thread(target=insert)
thread2=threading.Thread(target=arbitrage)
#set flag for calculation
thread1.start()

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/haricoolharan/anaconda3/envs/tensorflow/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/haricoolharan/anaconda3/envs/tensorflow/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-00cc4a3c98e0>", line 12, in insert
AttributeError: 'numpy.float64' object has no attribute 'append'


thread1
thread2
buy at 0.00059193 2021-03-02 21:04:00.030678
change length of the candle is        open      high       low     close
0  0.000592  0.000592  0.000592  0.000592
